In [0]:
CATALOGO = "v_credit"
SCHEMA_ORIGEM = "gold"
SCHEMA_DESTINO = "curated"
NOME_VIEW = "vw_performance_equipe"

full_view_name = f"{CATALOGO}.{SCHEMA_DESTINO}.{NOME_VIEW}"

In [0]:
query = f"""
CREATE OR REPLACE VIEW {full_view_name} AS
SELECT
    atendente.nm_atendente,
    atendente.ds_perfil, -- Especialista vs Generalista
    
    -- Volumetria
    COUNT(fato.pk_fato_atendimento) AS chamados_atendidos,
    
    -- Financeiro (NOVO!)
    SUM(fato.val_custo) AS custo_total_gerado,
    AVG(fato.val_custo) AS custo_medio_por_atendimento,
    
    -- Eficiência
    AVG(fato.val_tempo_atendimento) / 60 AS tma_medio_minutos,
    AVG(fato.val_tempo_espera) AS espera_media_segundos,
    
    -- Qualidade
    AVG(fato.val_nota_csat) AS nota_media_cliente,
    
    -- Taxa de Resolução
    (SUM(cast(fato.st_resolvido as double)) / COUNT(*)) * 100 AS taxa_resolucao_percentual

FROM {CATALOGO}.{SCHEMA_ORIGEM}.ft_atendimento_geral AS fato
JOIN {CATALOGO}.{SCHEMA_ORIGEM}.dm_atendente AS atendente ON fato.cd_atendente = atendente.cd_atendente

-- Filtro: Apenas Humanos (exclui robôs se houver na dimensão)
WHERE atendente.ds_perfil <> 'Robô'
GROUP BY 1, 2;
"""

spark.sql(query)
print(f"✅ View {full_view_name} criada/atualizada com sucesso!")